<a href="https://colab.research.google.com/github/josenomberto/UTEC-CDIAV3-PROG101/blob/main/Hackhaton_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from datetime import date, time
from typing import Optional


def generar_id(df: pd.DataFrame) -> str:
    if df.empty:
        return "1001"
    return str(df["IDCita"].astype(int).max() + 1)


def listar_citas(df):
  '''
  Funcion para listar las citas programadas
  '''
  print('\n')
  print(f"Lista de Citas:")
  print('-'*16)
  # for index, row in df.iterrows():
  #  print(f"Paciente: {row['Paciente']}")
  print(df)

def buscar_cita_por_paciente(df, paciente):
  '''
  Funcion para buscar las citas de un paciente en especifico
  '''
  print('\n')
  print(f'Busqueda de Citas:')
  print('-'*16)
  print(f"Citas del paciente {paciente}:")
  print('\n')
  if df[df['Paciente'] == paciente].empty:
    print(f"No se encontraron citas para el paciente {paciente}.")
  else:
    print(df[df['Paciente'] == paciente])

def buscar_cita_por_fecha(df, datos):
  '''
  Funcion para buscar las citas por fecha
  '''


def agregar_cita(df, paciente, fecha, hora, profesional, tratamiento):
    try:
        h, m = map(int, hora.split(':'))
        if not (8 <= h < 20) or m not in (0, 30):
            print("Error: horario de atención de 8am a 8pm (solo en puntos de 00 o 30).")
            return df
        fecha_hora = pd.to_datetime(f"{pd.to_datetime(fecha).strftime('%Y-%m-%d')} {hora}")
    except:
        print("Error: Formato de hora inválido. Use HH:MM (ej. 10:30, 14:00).")
        return df

    # Validar disponibilidad
    citas = df[df['Profesional'] == profesional].copy()
    if not citas.empty:
        citas['FH'] = pd.to_datetime(citas['Fecha'].dt.strftime('%Y-%m-%d') + ' ' + citas['Hora'])
        if fecha_hora in citas['FH'].values:
            print(f"Error: El profesional '{profesional}' ya tiene una cita a las {hora} del {fecha}.")
            return df

    # Calcular nuevo ID sin alterar el tipo original del DataFrame
    id_temp = pd.to_numeric(df['IDCita'], errors='coerce') if 'IDCita' in df.columns else pd.Series()
    nuevo_id = int(id_temp.max()) + 1 if not id_temp.empty and id_temp.notna().any() else 1

    nueva_fila = {
        'IDCita': nuevo_id,
        'Paciente': paciente,
        'Fecha': pd.to_datetime(fecha),
        'Hora': hora,
        'Profesional': profesional,
        'Tratamiento': tratamiento
    }

    df_nuevo = pd.DataFrame([nueva_fila])
    df = pd.concat([df, df_nuevo], ignore_index=True)
    print(f"Cita agregada exitosamente con ID '{nuevo_id}'.")
    return df


def modificar_cita(df, id_cita, nuevos_datos):
  '''
  Funcion para modificar cita existente
  '''
  if id_cita not in df['IDCita'].values:
      print(f"Error: La cita con ID '{id_cita}' no existe.")
      return df

  i = df[df['IDCita'] == id_cita].index[0]
  actual = df.loc[i].to_dict()
  nuevos = {**actual, **nuevos_datos}

  fecha = pd.to_datetime(nuevos.get('Fecha', actual['Fecha']))
  hora = nuevos.get('Hora', actual['Hora'])
  prof = nuevos.get('Profesional', actual['Profesional'])

  # Validar formato y rango horario
  try:
      h, m = map(int, hora.split(':'))
      if not (8 <= h < 20) or m not in (0, 30):
          print("Error: horario de atención de 8am a 8pm (solo en puntos de 00 o 30).")
          return df
      fh = pd.to_datetime(f"{fecha.strftime('%Y-%m-%d')} {hora}")
  except:
      print(f"Error: Hora inválida: {hora}. Formato esperado HH:MM (ej. 11:30, 12:00).")
      return df

  citas = df[(df['Profesional'] == prof) & (df['IDCita'] != id_cita)].copy()
  if not citas.empty:
      citas['FH'] = pd.to_datetime(citas['Fecha'].dt.strftime('%Y-%m-%d') + ' ' + citas['Hora'])
      if fh in citas['FH'].values:
          print(f"Error: El profesional '{prof}' ya tiene una cita a las {hora} del {fecha.strftime('%Y-%m-%d')}.")
          return df

  for k, v in nuevos_datos.items():
      if k == 'IDCita':
          print("Advertencia: El 'IDCita' no puede ser modificado.")
      else:
          df.loc[i, k] = pd.to_datetime(v) if k == 'Fecha' else v

  print(f"Cita con ID '{id_cita}' modificada exitosamente.")
  return df



def menu():
  '''
  Funcion para presentar menu de opciones del sistema de gestion de citas
  '''
  print('\n')
  print("SISTEMA DE GESTION DE CITAS")
  print("="*30)
  print("\t1. Listar Citas")
  print("\t2. Buscar Cita por Paciente")
  print("\t3. Buscar Cita por Fecha")
  print("\t4. Agregar Cita")
  print("\t5. Modificar Cita")
  print("\t9. Salir")
  opcion = input("Seleccione una opción: ")
  return opcion


# Leer informacion de archivo CSV
## df = pd.read_csv('dataframe.csv', delimiter=';', encoding ='latin-1')

# Crear datos ficticios de atención médica
data = {
    'IDCita': [
        '1001', '1002', '1003',
        '1004', '1005', '1006',
        '1007', '1008', '1009', '1010'
    ],
    'Paciente': [
        'Sofía Vargas', 'Alejandro Soto', 'María Fernanda Cruz',
        'Juan Pablo Rojas', 'Daniela Pérez', 'Ricardo Morales',
        'Emilia Guzmán', 'Gabriel Díaz', 'Valentina Ruiz', 'Felipe Mendoza'
    ],
    'Fecha': [
        '2025-07-10', '2025-07-10', '2025-07-11', '2025-07-11',
        '2025-07-12', '2025-07-12', '2025-07-13', '2025-07-13',
        '2025-07-14', '2025-07-14'
    ],
    'Hora': [
        '09:00', '10:30', '11:15', '14:00',
        '08:45', '16:00', '09:30', '15:00',
        '10:00', '13:30'
    ],
    'Profesional': [
        'Dra. Ana López', 'Dr. Carlos Ramírez', 'Lic. Paula Gómez',
        'Dra. Laura Castro', 'Dr. Miguel Torres', 'Lic. Valeria Salas',
        'Dr. Andrés Medina', 'Dra. Florencia Vera', 'Lic. Esteban Luna', 'Dr. Sebastián Gil'
    ],
    'Tratamiento': [
        'Consulta General', 'Control Odontológico', 'Sesión de Fisioterapia',
        'Ecografía Abdominal', 'Chequeo de Glicemia', 'Terapia Ocupacional',
        'Examen de la Vista', 'Inmunización', 'Asesoramiento Nutricional', 'Extracción de Sangre'
    ]
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Convertir la columna 'Fecha' a tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

# LLamado las funciones segun opcion elegida en menu
while True:
  opcion = menu()
  if opcion == '1':
    listar_citas(df)
  elif opcion == '2':
    paciente = input("Ingrese el nombre del paciente: ")
    buscar_cita_por_paciente(df, paciente)
  elif opcion == '3':
    fecha = input("Ingrese la fecha: ")
    buscar_cita_por_fecha(df, fecha)
  elif opcion == '4':
    nuevo_paciente     = input("Nombre del paciente: ").strip()
    nueva_fecha  = input("Fecha (YYYY-MM-DD): ").strip()
    nueva_hora   = input("Hora (HH:MM): ").strip()
    nuevo_profesional  = input("Profesional: ").strip()
    nuevo_tratamiento  = input("Tratamiento: ").strip()
    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df["Hora"] = pd.to_datetime(df["Hora"], format="%H:%M", errors="coerce")
    df = agregar_cita(df, nuevo_paciente, nueva_fecha, nueva_hora, nuevo_profesional, nuevo_tratamiento)
  elif opcion == '5':
    ID           = input("ID: ").strip()
    paciente     = input("Nombre del paciente: ").strip()
    fecha_input  = input("Fecha (YYYY-MM-DD): ").strip()
    hora_input   = input("Hora (HH:MM): ").strip()
    profesional  = input("Profesional: ").strip()
    tratamiento  = input("Tratamiento: ").strip()
    nuevos_datos= {'Paciente': paciente,
      'Fecha': fecha_input,
      'Hora': hora_input,
      'Profesional': profesional,
      'Tratamiento': tratamiento}
    df = modificar_cita(df, ID, nuevos_datos)
  elif opcion == '9':
    print("¡Hasta luego!")
    break
  else:
    print("Opción inválida. Por favor, seleccione una opción válida.")




SISTEMA DE GESTION DE CITAS
	1. Listar Citas
	2. Buscar Cita por Paciente
	3. Buscar Cita por Fecha
	4. Agregar Cita
	5. Modificar Cita
	9. Salir
Seleccione una opción: 1


Lista de Citas:
----------------
  IDCita             Paciente      Fecha   Hora          Profesional  \
0   1001         Sofía Vargas 2025-07-10  09:00       Dra. Ana López   
1   1002       Alejandro Soto 2025-07-10  10:30   Dr. Carlos Ramírez   
2   1003  María Fernanda Cruz 2025-07-11  11:15     Lic. Paula Gómez   
3   1004     Juan Pablo Rojas 2025-07-11  14:00    Dra. Laura Castro   
4   1005        Daniela Pérez 2025-07-12  08:45    Dr. Miguel Torres   
5   1006      Ricardo Morales 2025-07-12  16:00   Lic. Valeria Salas   
6   1007        Emilia Guzmán 2025-07-13  09:30    Dr. Andrés Medina   
7   1008         Gabriel Díaz 2025-07-13  15:00  Dra. Florencia Vera   
8   1009       Valentina Ruiz 2025-07-14  10:00    Lic. Esteban Luna   
9   1010       Felipe Mendoza 2025-07-14  13:30    Dr. Sebastián Gil   


KeyboardInterrupt: Interrupted by user